## Imports

In [1]:
pip install opencv-python==3.4.2.17

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install opencv-contrib-python==3.4.2.17

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install imutils

In [5]:
import imutils

In [6]:
# Task1 Hint: (with sample code for the SIFT detector)
# Initialize SIFT detector, detect keypoints, store and show SIFT keypoints of original image in a Numpy array
# Define parameters for SIFT initializations such that we find only 10% of keypoints
import cv2
import matplotlib.pyplot as plt

class SiftDetector():
    def __init__(self, norm="L2", params=None):
        self.detector=self.get_detector(params)
        self.norm=norm

    def get_detector(self, params):
        if params is None:
            params={}
            params["n_features"]=0
            params["n_octave_layers"]=3
            params["contrast_threshold"]=0.03
            params["edge_threshold"]=10
            params["sigma"]=1.6

        detector = cv2.xfeatures2d.SIFT_create(
                nfeatures=params["n_features"],
                nOctaveLayers=params["n_octave_layers"],
                contrastThreshold=params["contrast_threshold"],
                edgeThreshold=params["edge_threshold"],
                sigma=params["sigma"])

        return detector



# Task2 Hint:
# Upscale the image, compute SIFT features for rescaled image
# Apply BFMatcher with defined params and ratio test to obtain good matches, and then select and draw best 5 matches

# Task3 Hint: (with sampe code for the rotation)
# Rotate the image and compute SIFT features for rotated image
# Apply BFMatcher with defined params and ratio test to obtain good matches, and then select and draw best 5 matches
import math
import numpy as np
import sys

# image: image to rotate
# x:     x-coordinate of point we wish to rotate around
# y:     y-coordinate of point we wish to rotate around
# angle: degrees to rotate image by
#
# Returns a rotated copy of the original image
def rotate(image, x, y, angle):
    rot_matrix = cv2.getRotationMatrix2D((x, y), angle, 1.0)
    h, w = image.shape[:2]

    return cv2.warpAffine(image, rot_matrix, (w, h))

# Get coordinates of center point.
#
# image:  Image that will be rotated
# return: (x, y) coordinates of point at center of image
def get_img_center(image):
    height, width = image.shape[:2]
    center = height // 2, width // 2

    return center


## Import image in gray scale

In [7]:
img = cv2.imread('COMP9517_20T2_Lab2_Image.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

## Task 1, part A

In [8]:
def task1_a(img, name):
    sift = SiftDetector()
    detector = sift.get_detector(None)
    
    kp = detector.detect(img, None)
    img = cv2.drawKeypoints(img,kp,img)
    cv2.imwrite(name, img)


task1_a(img.copy(), 'task1_A.jpg')

### Settings for part B

I have experimented with both the approaches with trail and error. The n features method is straight simple and we just have to give N. For the contrast threshold, it will depend on the picture

In [9]:
def get_sift_detector():
    sift = SiftDetector()
    params = {}
    params["n_features"] = 0
    params["n_octave_layers"] = 3
    params["contrast_threshold"] = 0.136
    params["edge_threshold"] = 10
    params["sigma"] = 1.6
    detector = sift.get_detector(params)
    return detector

def get_sift_detector_n():
    sift = SiftDetector()
    params = {}
    params["n_features"] = 620
    params["n_octave_layers"] = 3
    params["contrast_threshold"] = 0.03
    params["edge_threshold"] = 10
    params["sigma"] = 1.6
    detector = sift.get_detector(params)
    return detector


## Task 1, part B

In [10]:
def task1_b(img, name):
    detector = get_sift_detector()

    kp = detector.detect(img, None)
    img = cv2.drawKeypoints(img, kp, img)
    cv2.imwrite(name, img)
    return img

    
task1_b(img.copy(), 'task1_B.jpg')

array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [241, 241, 241],
        [240, 240, 240],
        [239, 239, 239]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [240, 240, 240],
        [239, 239, 239],
        [239, 239, 239]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [239, 239, 239],
        [239, 239, 239],
        [239, 239, 239]],

       ...,

       [[107, 107, 107],
        [110, 110, 110],
        [119, 119, 119],
        ...,
        [121, 121, 121],
        [120, 120, 120],
        [120, 120, 120]],

       [[106, 106, 106],
        [108, 108, 108],
        [110, 110, 110],
        ...,
        [121, 121, 121],
        [120, 120, 120],
        [121, 121, 121]],

       [[104, 104, 104],
        [104, 104, 104],
        [112, 112, 112],
        ...,
        [118, 118, 118],
        [118, 118, 118],
        [121, 121, 121]]

## Task 2

In [18]:
def task2(img, name):
    # part a
    scale_percent = 115 # percent of original size
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    # resize image
    resized = cv2.resize(img, dim)
    
    # part b: increased to 710 from 625
    resized_image = task1_b(resized, name+'_B.jpg')

    # part c
    '''
    Yes, the key points are roughly the same as SIFT is scale independent. It only looks at the texture of objects.
    '''

    # part d
    detector = get_sift_detector()

    # find the keypoints and descriptors with SIFT
    kp1, des1 = detector.detectAndCompute(img, None)
    kp2, des2 = detector.detectAndCompute(resized, None)

    # create BFMatcher object
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
    matches = bf.knnMatch(des1, des2, k=2)

    # Apply ratio test
    good = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good.append([m])

    good = sorted(good, key=lambda x: x[0].distance)

    # cv2.drawMatchesKnn expects list of lists as matches.
    img3 = cv2.drawMatchesKnn(img, kp1, resized, kp2, good[:5], img, flags=2)
    cv2.imwrite(name+'_D.jpg', img3)
    

task2(img.copy(), 'task2')

## Task 3

In [23]:
def task3(img, name):
    # part a
    rotated = imutils.rotate_bound(img, angle=60)

    # part b
    detector = get_sift_detector()
    kp2, des2 = detector.detectAndCompute(rotated, None)

    rotated_dist = cv2.drawKeypoints(rotated, kp2, rotated)
    cv2.imwrite(name + '_B.jpg', rotated_dist)
    
    # part c
    '''
    Yes, the key points are roughly the same as SIFT is orientation independent. It only looks at the texture of objects. But we can see little variations. 
    '''
    
    # part d
    kp1, des1 = detector.detectAndCompute(img, None)

    # create BFMatcher object
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
    matches = bf.knnMatch(des1, des2, k=2)

    # Apply ratio test
    good = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good.append([m])

    good = sorted(good, key=lambda x: x[0].distance)

    # cv2.drawMatchesKnn expects list of lists as matches.
    img3 = cv2.drawMatchesKnn(img, kp1, rotated, kp2, good[:7], img, flags=2)
    cv2.imwrite(name+'_D.jpg', img3)
    

task3(img.copy(), 'task3')